In [1]:
## Data Collection: Load csv files and check contents

import pandas as pd
import glob

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)

# Define the path to your CSV files
csv_files = glob.glob('fuel_prices_*.csv')  # Assumes all files start with 'fuel_price_' and are in the current directory

# Load all CSV files into a list of DataFrames
df_list = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames into one DataFrame
df = pd.concat(df_list, ignore_index=True)

# Display the shape of the combined DataFrame to verify
print(df.shape)
print(df.head(5))
print(df.columns)

(469336, 12)
     SiteId         Site_Name Site_Brand Sites_Address_Line_1 Site_Suburb   
0  61401007  7-Eleven Coomera   7 Eleven      Pacific Highway     Coomera  \
1  61401007  7-Eleven Coomera   7 Eleven      Pacific Highway     Coomera   
2  61401007  7-Eleven Coomera   7 Eleven      Pacific Highway     Coomera   
3  61401007  7-Eleven Coomera   7 Eleven      Pacific Highway     Coomera   
4  61401007  7-Eleven Coomera   7 Eleven      Pacific Highway     Coomera   

  Site_State  Site_Post_Code  Site_Latitude  Site_Longitude       Fuel_Type   
0        QLD            4209     -27.868591      153.314206             e10  \
1        QLD            4209     -27.868591      153.314206             e10   
2        QLD            4209     -27.868591      153.314206             LPG   
3        QLD            4209     -27.868591      153.314206  Premium Diesel   
4        QLD            4209     -27.868591      153.314206     PULP 98 RON   

   Price TransactionDateutc  
0   1879   31/03/20

In [2]:
import pandas as pd

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)        # Set display width to avoid line breaks

# Step 1: Ensure 'TransactionDateutc' is in datetime64[ns] format
df['TransactionDateutc'] = pd.to_datetime(df['TransactionDateutc'], errors='coerce')

# Normalize 'TransactionDateutc' to midnight
df['TransactionDateutc'] = df['TransactionDateutc'].dt.normalize()

# Step 2: Aggregate data by 'SiteId' and 'TransactionDateutc' to remove duplicates
df_aggregated = df.groupby(['SiteId', 'TransactionDateutc']).agg({
    'Site_Name': 'first',  # Keep the first occurrence
    'Site_Brand': 'first',
    'Sites_Address_Line_1': 'first',
    'Site_Suburb': 'first',
    'Site_State': 'first',
    'Site_Post_Code': 'first',
    'Site_Latitude': 'first',
    'Site_Longitude': 'first',
    'Fuel_Type': 'first',
    'Price': 'mean'
}).reset_index()

# Step 3: Get all unique 'SiteId's
all_site_ids = df_aggregated['SiteId'].unique()

# Step 4: Create a full date range for weekly intervals
full_weeks = pd.date_range(start=df_aggregated['TransactionDateutc'].min(), 
                           end=df_aggregated['TransactionDateutc'].max(), 
                           freq='W').normalize()

# Step 5: Create an empty DataFrame with all combinations of 'SiteId' and week
full_index = pd.MultiIndex.from_product([all_site_ids, full_weeks], names=['SiteId', 'TransactionDateutc'])
empty_df = pd.DataFrame(index=full_index).reset_index()

# Normalize 'TransactionDateutc' in 'empty_df' to midnight
empty_df['TransactionDateutc'] = pd.to_datetime(empty_df['TransactionDateutc']).dt.normalize()

# Step 6: Resample the df_aggregated to match weekly intervals, grouped by 'SiteId'
df_aggregated.set_index('TransactionDateutc', inplace=True)

# Resample weekly and forward fill, ensuring 'SiteId' remains as a column
# Avoid resetting 'SiteId' index here
resampled_df = df_aggregated.groupby('SiteId', group_keys=False).resample('W').ffill().reset_index(drop=False)

# **Ensure 'SiteId' is already a column and doesn't need to be added again**
# Step 7: Normalize 'TransactionDateutc' in 'resampled_df'
resampled_df['TransactionDateutc'] = resampled_df['TransactionDateutc'].dt.normalize()

# Step 8: Merge 'resampled_df' with 'empty_df' on 'SiteId' and 'TransactionDateutc'
merged_df = pd.merge(empty_df, resampled_df, on=['SiteId', 'TransactionDateutc'], how='left')

# Step 9: Forward fill any remaining missing data (prices, brands, etc.)
merged_df = merged_df.groupby('SiteId', as_index=False).apply(lambda group: group.ffill()).reset_index(drop=True)


print(merged_df)

# Step 10: Verify that 'SiteId' is present and check the output
print(merged_df[['SiteId', 'TransactionDateutc', 'Site_Brand']].head())  # Verify SiteId is present

/var/folders/wz/zmc0gwmn2n57xys67hh2y13c0000gn/T/ipykernel_2070/3726965435.py:8: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['TransactionDateutc'] = pd.to_datetime(df['TransactionDateutc'], errors='coerce')


         SiteId TransactionDateutc      Site_Name     Site_Brand Sites_Address_Line_1 Site_Suburb Site_State  Site_Post_Code  Site_Latitude  Site_Longitude Fuel_Type        Price
0      61290151         2023-12-31            NaN            NaN                  NaN         NaN        NaN             NaN            NaN             NaN       NaN          NaN
1      61290151         2024-01-07            NaN            NaN                  NaN         NaN        NaN             NaN            NaN             NaN       NaN          NaN
2      61290151         2024-01-14  Liberty Surat        Liberty   61 Burrowes Street       Surat        QLD          4417.0     -27.151687      149.067742    Diesel  1965.666667
3      61290151         2024-01-21  Liberty Surat        Liberty   61 Burrowes Street       Surat        QLD          4417.0     -27.151687      149.067742    Diesel  1965.666667
4      61290151         2024-01-28  Liberty Surat        Liberty   61 Burrowes Street       Surat        

In [3]:
# Filter the data for a specific week and brand (e.g., most recent week and Liberty brand)
week_to_visualize = merged_df['TransactionDateutc'].max()  # Most recent week
brand_to_visualize = 'United'  # Replace with any brand you want

# Filter for the specific week and brand
filtered_map_data = merged_df[
    (merged_df['TransactionDateutc'] == week_to_visualize) &
    (merged_df['Site_Brand'] == brand_to_visualize)
]

# Check the filtered data
print(filtered_map_data[['SiteId', 'Site_Brand', 'Site_Latitude', 'Site_Longitude', 'Price']].head())

         SiteId Site_Brand  Site_Latitude  Site_Longitude        Price
2879   61401180     United     -27.532891      152.983297  1637.500000
8079   61401505     United     -27.655108      152.747776  1680.666667
12959  61401773     United     -27.666991      153.139183  1602.333333
14039  61401816     United     -26.622451      153.042205  1701.500000
14119  61401819     United     -26.527951      153.090382  1767.500000


In [4]:
import folium

# Initialize a folium map centered on Australia
m = folium.Map(location=[-27.4705, 153.0260], zoom_start=10)  # Brisbane coordinates with appropriate zoom

# Add fuel station markers to the map
for idx, row in filtered_map_data.iterrows():
    folium.CircleMarker(
        location=[row['Site_Latitude'], row['Site_Longitude']],
        radius=5,  # Adjust the size of the marker
        popup=f"Brand: {row['Site_Brand']}<br>Price: {row['Price']}",
        color='blue',  # Customize color based on brand or other metrics
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

# Save the map to an HTML file or display it directly
m.save('fuel_stations_map.html')  # Save to an HTML file
m  # Display the map (useful in Jupyter notebooks)